In [1]:
# from flask import Flask, render_template, redirect
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from flask_sqlalchemy import SQLAlchemy

In [2]:
engine = create_engine("sqlite:///Data/belly_button_biodiversity.sqlite")
inspector = inspect(engine)

In [4]:
def find_col_names(eng, table_name):
    inspector = inspect(eng)
    tables = {}
    
    for table in inspector.get_table_names():
        columns = inspector.get_columns(table)
        names = []
        for c in columns:
            names.append(c['name'])
            tables[table] = names
    
    column_names = tables[table_name]        
    del column_names[:1] #del first name(index)
    
    return column_names

In [7]:
tables = {}
    
for table in inspector.get_table_names():
    columns = inspector.get_columns(table)
    names = []
    for c in columns:
        names.append(c['name'])
        tables[table] = names

In [ ]:
Base = automap_base()
Base.prepare(engine, reflect=True)

In [ ]:
Base.classes.keys()

In [ ]:
Samples_metadata = Base.classes.samples_metadata
Samples = Base.classes.samples
Otu = Base.classes.otu

In [ ]:
session = Session(engine)

In [ ]:
bact = session.query(Otu.lowest_taxonomic_unit_found).all()

In [ ]:
# labels, values = zip(*lyrics.items())  check it https://docs.python.org/3/library/functions.html#zip

In [ ]:
from flask import Flask, render_template, redirect, jsonify, request
from sqlalchemy import inspect
from sqlalchemy.ext.automap import automap_base
from flask_sqlalchemy import SQLAlchemy

In [ ]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///Data/belly_button_biodiversity.sqlite"

db = SQLAlchemy(app)

db.metadata.bind = db.engine
db.metadata.reflect(db.engine)

Base = automap_base(metadata=db.metadata)
Base.prepare()

In [ ]:
Base.classes.keys()

In [ ]:
Samples_metadata = Base.classes.samples_metadata
Samples = Base.classes.samples
Otu = Base.classes.otu